In [2]:
import requests
import pandas as pd
import numpy as np
import io
import geopandas as gpd
from datetime import timedelta
from shapely.geometry import Point

In [3]:
import os

In [4]:
gdf_thailand = gpd.read_file(r'E:\_CoopProject\data\thailand_gis-main\tambon\thailand_province_amphoe_tambon_simplify')
gdf_thailand

,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49453 13.75759, 100.49494 13.757..."
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.74800, 100.50412 13.747..."
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50131 13.74800, 100.49633 13.748..."
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.50554 13.75378, 100.50412 13.747..."
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,Thailand,ประเทศไทย,TH,"POLYGON ((100.49875 13.75560, 100.49821 13.751..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7420,Phadung Mat,ผดุงมาตร,TH961203,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.71286 6.17046, 101.71403 6.16538..."
7421,Chang Phueak,ช้างเผือก,TH961204,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.67997 5.99753, 101.67567 5.98895..."
7422,Chuap,จวบ,TH961301,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.86883 6.29824, 101.86908 6.29436..."
7423,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,Thailand,ประเทศไทย,TH,"POLYGON ((101.88570 6.19545, 101.87473 6.19872..."


In [5]:
def getData(date):
    # get from url
    y, m, d = date.split('-')
    url = f'https://firms2.modaps.eosdis.nasa.gov/api/country/csv/cea584916717f2288aa22711dadd89e7/MODIS_NRT/THA/2/{y}-{m}-{int(d)-1}'
    urlData = requests.get(url).content
    fire_detect = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    gdf = gpd.GeoDataFrame(fire_detect)

    # edit time column
    gdf['acq_time'] = gdf['acq_time'].astype('str')
    gdf['acq_time'] = gdf['acq_time'].apply(lambda x: x.rjust(4, '0'))

    gdf['acq_date_time'] = pd.to_datetime(gdf['acq_date'] + ' ' + gdf['acq_time'])
    gdf['th_date_time'] = gdf['acq_date_time'] + timedelta(hours=7)
    gdf = gdf[(gdf['th_date_time']>=date) & (gdf['th_date_time']<f'{y}-{m}-{int(d)+1}')].reset_index().drop(columns=['index', 'acq_date_time'])
    gdf['th_date'] = gdf['th_date_time'].apply(lambda x: str(x).split(' ')[0])
    gdf['th_time'] = gdf['th_date_time'].apply(lambda x: str(x).split(' ')[1])

    # Create Point from lat, long
    geometry = [Point(xy) for xy in zip(gdf['longitude'], gdf['latitude'])]
    gdf['geometry'] = geometry
    
    return gpd.GeoDataFrame(gdf, crs=4326)

date_str = '2023-05-22'
data = getData(date_str)
data = data.loc[:, ['latitude', 'longitude', 'confidence', 'th_date', 'th_time', 'geometry']]
data

,latitude,longitude,confidence,th_date,th_time,geometry
0,17.46168,101.89087,60,2023-05-22,10:45:00,POINT (101.89087 17.46168)
1,17.65386,100.00925,68,2023-05-22,10:45:00,POINT (100.00925 17.65386)
2,18.24682,99.53375,88,2023-05-22,10:45:00,POINT (99.53375 18.24682)
3,18.58789,99.05137,31,2023-05-22,10:45:00,POINT (99.05137 18.58789)
4,18.69384,100.23157,62,2023-05-22,10:45:00,POINT (100.23157 18.69384)
5,19.67976,100.30830,48,2023-05-22,10:45:00,POINT (100.30830 19.67976)
6,19.77347,99.76343,64,2023-05-22,10:45:00,POINT (99.76343 19.77347)
7,19.83875,99.63877,51,2023-05-22,10:45:00,POINT (99.63877 19.83875)
8,10.91775,99.47620,63,2023-05-22,10:47:00,POINT (99.47620 10.91775)
9,10.91955,99.46468,69,2023-05-22,10:47:00,POINT (99.46468 10.91955)


In [6]:
new_data = gpd.overlay(data, gdf_thailand, how='intersection')
new_data.head()

,latitude,longitude,confidence,th_date,th_time,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,ADM0_EN,ADM0_TH,ADM0_PCODE,geometry
0,17.46168,101.89087,60,2023-05-22,10:45:00,Na Din Dam,นาดินดำ,TH420109,Mueang Loei,เมืองเลย,TH4201,Loei,เลย,TH42,Thailand,ประเทศไทย,TH,POINT (101.89087 17.46168)
1,17.65386,100.00925,68,2023-05-22,10:45:00,Fai Luang,ฝายหลวง,TH530804,Laplae,ลับแล,TH5308,Uttaradit,อุตรดิตถ์,TH53,Thailand,ประเทศไทย,TH,POINT (100.00925 17.65386)
2,18.24682,99.53375,88,2023-05-22,10:45:00,Phrabat,พระบาท,TH520105,Mueang Lampang,เมืองลำปาง,TH5201,Lampang,ลำปาง,TH52,Thailand,ประเทศไทย,TH,POINT (99.53375 18.24682)
3,18.58789,99.05137,31,2023-05-22,10:45:00,Ban Klang,บ้านกลาง,TH510112,Mueang Lamphun,เมืองลำพูน,TH5101,Lamphun,ลำพูน,TH51,Thailand,ประเทศไทย,TH,POINT (99.05137 18.58789)
4,18.69384,100.23157,62,2023-05-22,10:45:00,Sa-Iap,สะเอียบ,TH540606,Song,สอง,TH5406,Phrae,แพร่,TH54,Thailand,ประเทศไทย,TH,POINT (100.23157 18.69384)


In [8]:
out_fc_path = os.path.join('C:\coop_project_fire_spot\Fire-Spot-Thailand\data', date_str)
os.makedirs(out_fc_path, exist_ok=True)
new_data.to_file(out_fc_path, encoding='utf-8')

In [16]:
prov = pd.read_excel(r'C:\coop_project_fire_spot\Fire-Spot-Thailand\data\all_tambons.xlsx', sheet_name='Sheet1', header=0)
prov

,ADM3_EN,ADM3_TH,ADM3_PCODE,ADM2_EN,ADM2_TH,ADM2_PCODE,ADM1_EN,ADM1_TH,ADM1_PCODE,Shape_Area
0,Phraborom Maharatchawang,พระบรมมหาราชวัง,TH100101,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000128
1,Wang Burapha Phirom,วังบูรพาภิรมย์,TH100102,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000061
2,Wat Ratchabophit,วัดราชบพิธ,TH100103,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000018
3,Samran Rat,สำราญราษฎร์,TH100104,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000019
4,San Chaopho Suea,ศาลเจ้าพ่อเสือ,TH100105,Phra Nakhon,พระนคร,TH1001,Bangkok,กรุงเทพมหานคร,TH10,0.000013
...,...,...,...,...,...,...,...,...,...,...
7420,Phadung Mat,ผดุงมาตร,TH961203,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,0.004115
7421,Chang Phueak,ช้างเผือก,TH961204,Chanae,จะแนะ,TH9612,Narathiwat,นราธิวาส,TH96,0.031049
7422,Chuap,จวบ,TH961301,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,0.004813
7423,Bu Kit,บูกิต,TH961302,Cho-Airong,เจาะไอร้อง,TH9613,Narathiwat,นราธิวาส,TH96,0.005915
